In [1]:
# Loading the libraries
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

In [2]:
data = pd.read_csv('Analysis_ready.csv')

/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.head(2)

,Unnamed: 0,STATE,AGE_YRS,SEX,DIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,NUMDAYS,V_ADMINBY,V_FUNDBY,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,VAX_MANU
0,0,TX,33.0,F,0,0,0,0,0.0,0,0,1,2.0,PVT,0,0,1,0,MODERNA
1,1,CA,73.0,F,0,0,0,0,0.0,0,0,1,0.0,SEN,0,0,1,0,MODERNA


In [4]:
data = data.replace([np.inf, -np.inf], 0)
data = data.drop(['Unnamed: 0'], axis =1)

In [5]:
data.shape

(343491, 18)

In [6]:
## Removing unwanted Column

data = data.drop(['V_ADMINBY'], axis = 1)

In [7]:
# Mortality rate

data['DIED'].value_counts()

0    339622
1      3869
Name: DIED, dtype: int64

In [8]:
obj = data.dtypes == np.object
print(obj)

STATE            True
AGE_YRS         False
SEX              True
DIED            False
L_THREAT        False
ER_VISIT        False
HOSPITAL        False
HOSPDAYS        False
X_STAY          False
DISABLE         False
RECOVD          False
NUMDAYS         False
V_FUNDBY         True
BIRTH_DEFECT    False
OFC_VISIT       False
ER_ED_VISIT     False
VAX_MANU         True
dtype: bool


In [9]:
dummydf = pd.DataFrame()   
for i in data.columns[obj]:
    print(i)
    dummy = pd.get_dummies(data[i], drop_first=True)
    dummydf = pd.concat([dummydf, dummy], axis=1)

STATE
SEX
V_FUNDBY
VAX_MANU


In [10]:
#Merge the dummy and dataset

data_1 = data
data_1 = pd.concat([data_1,dummydf], axis=1)

In [11]:
dummydf.head(10)

,AK,AL,AR,AS,AZ,CA,CO,CT,Ca,DC,...,XV,M,U,0,OTH,PUB,PVT,UNK,MODERNA,PFIZER\BIONTECH
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [12]:
obj1 = data_1.dtypes == np.object

In [13]:
obj1

STATE               True
AGE_YRS            False
SEX                 True
DIED               False
L_THREAT           False
                   ...  
PUB                False
PVT                False
UNK                False
MODERNA            False
PFIZER\BIONTECH    False
Length: 88, dtype: bool

In [14]:
data_1 = data_1.drop(data_1.columns[obj1], axis=1)

In [15]:
data_1.head(5)

,AGE_YRS,DIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,NUMDAYS,...,XV,M,U,0,OTH,PUB,PVT,UNK,MODERNA,PFIZER\BIONTECH
0,33.0,0,0,0,0,0.0,0,0,1,2.0,...,0,0,0,1,0,0,0,0,1,0
1,73.0,0,0,0,0,0.0,0,0,1,0.0,...,0,0,0,1,0,0,0,0,1,0
2,23.0,0,0,0,0,0.0,0,0,2,0.0,...,0,0,0,1,0,0,0,0,0,1
3,58.0,0,0,0,0,0.0,0,0,1,0.0,...,0,0,0,1,0,0,0,0,1,0
4,47.0,0,0,0,0,0.0,0,0,0,7.0,...,0,0,0,1,0,0,0,0,1,0


In [16]:
data_1.dtypes

AGE_YRS            float64
DIED                 int64
L_THREAT             int64
ER_VISIT             int64
HOSPITAL             int64
                    ...   
PUB                  uint8
PVT                  uint8
UNK                  uint8
MODERNA              uint8
PFIZER\BIONTECH      uint8
Length: 84, dtype: object

In [17]:
X = data_1.drop(['DIED'], axis=1)

In [18]:
Y = data_1['DIED']

In [19]:
## Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(240443, 83)
(240443,)
(103048, 83)
(103048,)


# Weight of Evidence

In [20]:
data_test = data_1.copy()

data_test = data_test.replace([np.inf, -np.inf], 0)

print("Number of 1's :", data_test.DIED.sum())

Number of 1's : 3869


In [21]:
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF = pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        d['% of Events'] = d['Events'] / d['Events'].sum()
        d['Non-Events'] = d['N'] - d['Events']
        d['% of Non-Events'] = d['Non-Events'] / d['Non-Events'].sum()
        d.loc[d['% of Non-Events'] == 0.0,'% of Non-Events'] = 1e-312
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], 
                            "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF
iv_table = iv_woe(data = data_test, target = 'DIED')

Information value of AGE_YRS is 1.432297
Information value of L_THREAT is 0.01255
Information value of ER_VISIT is inf
Information value of HOSPITAL is 0.208083
Information value of HOSPDAYS is 0.0
Information value of X_STAY is 0.001383


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of DISABLE is 0.001957
Information value of RECOVD is 3.143333
Information value of NUMDAYS is 0.55676
Information value of BIRTH_DEFECT is 0.000212
Information value of OFC_VISIT is 0.07528
Information value of ER_ED_VISIT is 0.048825
Information value of AK is 0.000962
Information value of AL is 0.000266
Information value of AR is 0.003041
Information value of AS is inf
Information value of AZ is 0.003898
Information value of CA is 0.017771
Information value of CO is 0.002519
Information value of CT is 0.000711


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of Ca is inf
Information value of DC is 0.000142
Information value of DE is 0.000599
Information value of FL is 0.001643
Information value of FM is inf
Information value of GA is 0.000707
Information value of GU is 0.000549
Information value of HI is 0.000155


...

/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of UT is 0.002066
Information value of VA is 0.001005
Information value of VI is inf
Information value of VT is 0.000486
Information value of WA is 0.001693
Information value of WI is 3e-06
Information value of WV is 0.00112
Information value of WY is 0.000137


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Information value of XB is inf
Information value of XV is inf
Information value of M is 0.261113
Information value of U is 0.004151
Information value of 0 is 0.016303
Information value of OTH is 0.004842
Information value of PUB is 9.4e-05
Information value of PVT is inf
Information value of UNK is 3.6e-05
Information value of MODERNA is 0.002044
Information value of PFIZER\BIONTECH is 0.005237


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
def WoE_transform(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF = pd.DataFrame()
    newDF = pd.concat([newDF,data], axis=1)
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE on all the independent variables
    for ivars in cols[~cols.isin([target])]:

        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
            
            d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
            d.columns = ['Cutoff', 'N', 'Events']
            d['% of Events'] = d['Events'] / d['Events'].sum()
            d['Non-Events'] = d['N'] - d['Events']
            d['% of Non-Events'] = d['Non-Events'] / d['Non-Events'].sum()
            d.loc[d['% of Non-Events'] == 0.0,'% of Non-Events'] = 1e-312
            d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])

            for i in range(d.shape[0]):
                interval = d.iloc[i]['Cutoff']
                left = interval.left
                right = interval.right
                IV_value = d.iloc[i]['WoE']
                newDF.loc[(data[ivars] > left) & (data[ivars] <= right),ivars] = IV_value
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})
            d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
            d.columns = ['Cutoff', 'N', 'Events']
            d['% of Events'] = d['Events'] / d['Events'].sum()
            d['Non-Events'] = d['N'] - d['Events']
            d['% of Non-Events'] = d['Non-Events'] / d['Non-Events'].sum()
            d.loc[d['% of Non-Events'] == 0.0,'% of Non-Events'] = 1e-312
            d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])

            for i in range(d.shape[0]):
                interval = d.iloc[i]['Cutoff']
                WoE_value = d.iloc[i]['WoE']
                newDF.loc[data[ivars] == interval,ivars] = WoE_value
    for ivars in cols:
        newDF[ivars] = newDF[ivars].astype('float64')
    return newDF
WoE_table = WoE_transform(data = data_test, target = 'DIED')

X = WoE_table.drop('DIED', axis=1)
Y = WoE_table['DIED']
print(X.shape)
print(Y.shape)

/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages

(343491, 83)
(343491,)


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=7)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

X_train = X_train.replace([np.inf, -np.inf], 0)
X_test = X_test.replace([np.inf, -np.inf], 0)

(240443, 83)
(240443,)
(103048, 83)
(103048,)


# Modelling with Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
c = [0.001,0.01,0.1,1.0,10.0,100.0]
num_cv_splits = 5

from sklearn.preprocessing import StandardScaler
z_scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_z = z_scaler.fit_transform(X_train)
X_test_z = z_scaler.transform(X_test)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
c = [0.001,0.01,0.1,1.0,10.0,100.0]
num_cv_splits = 5
kf = KFold(n_splits=num_cv_splits)
for C in c:
    auc = 0.0
    acc = 0.0
    for train_index, test_index in kf.split(X_train):
        X_train_cv, X_test_cv = X_train_z[train_index], X_train_z[test_index]
        Y_train_cv, Y_test_cv = Y_train.iloc[train_index], Y_train.iloc[test_index]
        clf = LogisticRegression(C=C, random_state=5)
        clf.fit(X_train_cv,Y_train_cv)
        acc += clf.score(X_test_cv,Y_test_cv)
        pred = clf.predict_proba(X_test_cv)[:,1]
        auc += roc_auc_score(y_true = Y_test_cv, y_score = pred)
    print('C =',C,'; mean accuracy =',acc/num_cv_splits,'; auc =',auc/num_cv_splits)

C = 0.001 ; mean accuracy = 0.9885794120491646 ; auc = 0.9294894748840884
C = 0.01 ; mean accuracy = 0.98864179663768 ; auc = 0.9381648517686243


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

C = 0.1 ; mean accuracy = 0.9885960474378145 ; auc = 0.9399832117006705


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

C = 1.0 ; mean accuracy = 0.9885918883960578 ; auc = 0.9398156177789458


/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/krishm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#log

C = 10.0 ; mean accuracy = 0.988587729354301 ; auc = 0.9395139314894984
C = 100.0 ; mean accuracy = 0.988587729354301 ; auc = 0.9396269673837858


# Modelling with Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score,roc_curve,confusion_matrix
depths = [3,4,5,6,7,8,9,10,11]

In [27]:
num_cv_splits = 5
kf = KFold(n_splits=num_cv_splits)
for depth in depths:
    auc = 0.0
    acc = 0.0
    for train_index, test_index in kf.split(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
        Y_train_cv, Y_test_cv = Y_train.iloc[train_index], Y_train.iloc[test_index]
        clf = DecisionTreeClassifier(max_depth=depth, random_state=5)
        clf.fit(X_train_cv,Y_train_cv)
        acc += clf.score(X_test_cv,Y_test_cv)
        pred = clf.predict_proba(X_test_cv)[:,1]
        auc += roc_auc_score(y_true = Y_test_cv, y_score = pred)
    print('depth =', depth, '; mean accuracy =', acc/num_cv_splits, '; auc =', auc/num_cv_splits)

depth = 3 ; mean accuracy = 0.9885128685053786 ; auc = 0.9050642135743112
depth = 4 ; mean accuracy = 0.9886334790730846 ; auc = 0.9183425705942225
depth = 5 ; mean accuracy = 0.9886210019478143 ; auc = 0.923881791497051
depth = 6 ; mean accuracy = 0.988645955938896 ; auc = 0.9317356185738145
depth = 7 ; mean accuracy = 0.988779043545386 ; auc = 0.9369435282657708
depth = 8 ; mean accuracy = 0.9887707265861948 ; auc = 0.935628463932465
depth = 9 ; mean accuracy = 0.9887041820910591 ; auc = 0.9329831629239578
depth = 10 ; mean accuracy = 0.9885752534398394 ; auc = 0.9304231598833084
depth = 11 ; mean accuracy = 0.9883880988094293 ; auc = 0.9256084696419095


# Modelling with Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
num_trees = [25,50,75,100,125,150,175,200,225,250]
max_depth = [3,4,5,6]
num_cv_splits = 5

In [29]:
kf = KFold(n_splits=num_cv_splits)
for tree in num_trees:
    for depth in max_depth:
        auc = 0.0
        acc = 0.0
        for train_index, test_index in kf.split(X_train):
            X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
            Y_train_cv, Y_test_cv = Y_train.iloc[train_index], Y_train.iloc[test_index]
            clf = RandomForestClassifier(n_estimators=tree, max_depth=depth, n_jobs = 8, random_state=5)
            clf.fit(X_train_cv,Y_train_cv)
            acc += clf.score(X_test_cv,Y_test_cv)
            pred = clf.predict_proba(X_test_cv)[:,1]
            auc += roc_auc_score(y_true = Y_test_cv, y_score = pred)
        print('num_trees =',tree,'; depth=',depth,'; mean accuracy =',acc/num_cv_splits,'; auc =',auc/num_cv_splits)

num_trees = 25 ; depth= 3 ; mean accuracy = 0.9885128685053786 ; auc = 0.9074575337532842
num_trees = 25 ; depth= 4 ; mean accuracy = 0.9885128685053786 ; auc = 0.9032016222906899
num_trees = 25 ; depth= 5 ; mean accuracy = 0.9885128685053786 ; auc = 0.9118238536922763
num_trees = 25 ; depth= 6 ; mean accuracy = 0.9885170275471353 ; auc = 0.9231355089769087
num_trees = 50 ; depth= 3 ; mean accuracy = 0.9885128685053786 ; auc = 0.9047749103124147
num_trees = 50 ; depth= 4 ; mean accuracy = 0.9885128685053786 ; auc = 0.9110459679431624
num_trees = 50 ; depth= 5 ; mean accuracy = 0.9885128685053786 ; auc = 0.9152264908004414
num_trees = 50 ; depth= 6 ; mean accuracy = 0.9885128685053786 ; auc = 0.9263023157233391
num_trees = 75 ; depth= 3 ; mean accuracy = 0.9885128685053786 ; auc = 0.9011595671267582
num_trees = 75 ; depth= 4 ; mean accuracy = 0.9885128685053786 ; auc = 0.9072791621935297
num_trees = 75 ; depth= 5 ; mean accuracy = 0.9885128685053786 ; auc = 0.9157533798501513
num_trees 

In [30]:
# Final model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
num_trees = 250
max_depth = 5
clf = RandomForestClassifier(n_estimators=num_trees, max_depth=max_depth, n_jobs = 4, random_state=5)
clf.fit(X_train,Y_train)
pred = clf.predict_proba(X_test)
auc = roc_auc_score(y_true = Y_test, y_score = pred[:,1])
print('num_trees =',num_trees,'; depth=',max_depth,'; auc =',auc)

num_trees = 250 ; depth= 5 ; auc = 0.9176759539967


# Modelling with LSTM

# Treatment of Class Imbalance - SMOTE

In [31]:
# check version number
import imblearn
from collections import Counter
print(imblearn.__version__)

0.8.1


In [32]:
# summarize class distribution
counter = Counter(Y_train)
print(counter)

Counter({0.0: 237681, 1.0: 2762})


In [33]:
# transform the dataset
oversample = SMOTE()
X_train, Y_train = oversample.fit_resample(X_train, Y_train)

In [34]:
# summarize class distribution
counter = Counter(Y_train)
print(counter)

Counter({0.0: 237681, 1.0: 237681})


In [35]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(83,)),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [36]:
model.fit(X_train, Y_train, epochs=50, batch_size=1)

Epoch 1/50
475362/475362 [==============================] - 637s 1ms/step - loss: 0.2240 - accuracy: 0.9120
Epoch 2/50
475362/475362 [==============================] - 960s 2ms/step - loss: 0.1895 - accuracy: 0.9277
Epoch 3/50
475362/475362 [==============================] - 615s 1ms/step - loss: 0.1754 - accuracy: 0.9331
Epoch 4/50
475362/475362 [==============================] - 630s 1ms/step - loss: 0.1654 - accuracy: 0.9371
Epoch 5/50
475362/475362 [==============================] - 630s 1ms/step - loss: 0.1607 - accuracy: 0.9389
Epoch 6/50
475362/475362 [==============================] - 640s 1ms/step - loss: 0.1579 - accuracy: 0.9397
Epoch 7/50
475362/475362 [==============================] - 625s 1ms/step - loss: 0.1556 - accuracy: 0.94110s - l
Epoch 8/50
475362/475362 [==============================] - 632s 1ms/step - loss: 0.1540 - accuracy: 0.9415
Epoch 9/50
475362/475362 [==============================] - 990s 2ms/step - loss: 0.1530 - accuracy: 0.9420
Epoch 10/50
475362/475

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test)

In [ ]:
print('Test accuracy:', test_acc)